In [4]:
!git clone https://github.com/AlexWan0/DomiKnowS.git
!cd DomiKnowS && git checkout notebook-program-name
!mv DomiKnowS/* .
!pip install -r requirements.txt

Cloning into 'DomiKnowS'...
remote: Enumerating objects: 15454, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 15454 (delta 104), reused 167 (delta 104), pack-reused 15283
Receiving objects: 100% (15454/15454), 706.57 MiB | 32.98 MiB/s, done.
Resolving deltas: 100% (11133/11133), done.
Checking out files: 100% (314/314), done.
Branch 'notebook-program-name' set up to track remote branch 'notebook-program-name' from 'origin'.
Switched to a new branch 'notebook-program-name'
     |████████████████████████████████| 23.7MB 130kB/s 
     |████████████████████████████████| 11.1MB 44.0MB/s 
     |████████████████████████████████| 804.1MB 23kB/s 
     |████████████████████████████████| 2.3MB 40.5MB/s 
     |████████████████████████████████| 245kB 49.2MB/s 
     |████████████████████████████████| 901kB 46.0MB/s 
     |████████████████████████████████| 3.3MB 41.1MB/s 
  Created wheel for acls: filename=acls-1.0.2-cp37-none-any

In [5]:
!pip install torchtext==0.9 --no-dependencies

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from regr.graph import Graph, Concept, Relation
from regr.graph.relation import disjoint

Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
  review = Concept(name='review')

  positive = review(name='positive')
  negative = review(name='negative')

  disjoint(positive, negative)

In [28]:
from regr.sensor.pytorch.sensors import ReaderSensor

review['text'] = ReaderSensor(keyword='text')

review[positive] = ReaderSensor(keyword='positive', label=True)
review[negative] = ReaderSensor(keyword='negative', label=True)

In [29]:
embed_size = 300
hidden_size = 100
num_classes = 2
drop_rate = 0.5

In [30]:
from regr.sensor.pytorch.sensors import FunctionalSensor
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import torch

class EmbeddingSensor(FunctionalSensor):
  def __init__(self, *pres, **kwarg):
    super().__init__(*pres, **kwarg)

    self.vocab = GloVe(name='840B', dim=embed_size)
    self.tokenizer = get_tokenizer('spacy', language='en')
  
  def forward(self, *inputs):
    text = inputs[0]

    tokens_batch = [self.tokenizer(text)]

    emb_batch = []
    for tokens in tokens_batch:
      rev_emb = torch.empty((len(tokens), embed_size))
      for i, tok in enumerate(tokens):
        rev_emb[i] = self.vocab[tok]
      
      emb_batch.append(rev_emb)

    padded = pad_sequence(emb_batch)

    out = padded.to(device=self.device)

    return out

In [31]:
review['text_embed'] = EmbeddingSensor('text')

INFO:torchtext.vocab:Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [32]:
from torch import nn

class LSTMModule(nn.Module):
  def __init__(self):
    super(LSTMModule, self).__init__()

    self.rnn = nn.LSTM(embed_size, hidden_size, bidirectional=True)
    self.dropout = nn.Dropout(p=drop_rate)

  def forward(self, input):
    output, (h, c) = self.rnn(input)
    forward, backward = torch.chunk(output, 2, dim=2)
    comb = torch.cat((forward[-1,:,:], backward[0,:,:]), dim=1)

    return self.dropout(comb)

In [33]:
from regr.sensor.pytorch.learners import ModuleLearner
from torch import nn

review['rnn_embed'] = ModuleLearner('text_embed', module=LSTMModule())

review[positive] = ModuleLearner('rnn_embed', module=nn.Linear(hidden_size * 2, num_classes))
review[negative] = ModuleLearner('rnn_embed', module=nn.Linear(hidden_size * 2, num_classes))

In [34]:
from regr.program import SolverPOIProgram
from regr.program.metric import MacroAverageTracker, PRF1Tracker, DatanodeCMMetric
from regr.program.loss import NBCrossEntropyLoss

program = SolverPOIProgram(graph, inferTypes=['ILP', 'local/argmax'], loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})

In [35]:
from torchtext.datasets import IMDB
import os

if not os.path.exists('/content/.data/aclImdb/test/neg/127_3.txt'):
  IMDB()

In [36]:
import logging

logging.basicConfig(level=logging.INFO)

In [37]:
import glob
import random

def get_data(directory, label):
  data_all = []
  for path in glob.glob(os.path.join(directory, label + '/*.txt')):
    data_dict = {}
    with open(path, 'r') as f:
      data_dict['text'] = f.read()
      data_dict['positive'] = [1 if label == 'pos' else 0]
      data_dict['negative'] = [1 if label == 'neg' else 0]
    data_all.append(data_dict)
  return data_all

train_data = get_data('.data/aclImdb/train', 'pos')
train_data.extend(get_data('.data/aclImdb/train', 'neg'))
random.shuffle(train_data)

test_data = get_data('.data/aclImdb/test', 'pos')
test_data.extend(get_data('.data/aclImdb/test', 'neg'))
random.shuffle(test_data)

In [38]:
print(len(train_data))
print(train_data[0])

25000
{'text': "Lame, cliched superhero action movie drivel. I had high hopes for this movie, and the genre of HK buddy cop actioneers is one that i don't despise, but very rarely do i see a storyline as trite and ludicrous as this one was. This would have been forgivable, as it always is in these kinds of movies, when the action compensates, unfortunately, it did not. The action does carry the trademark surreality and over the top nature of HK action, but it's not very involving, obscenely gory, and in fact often completely incoherent (perhaps this is due to re-editing for american release, it does show signs in many places of patchwork). I was very disappointed.", 'positive': [0], 'negative': [1]}


In [39]:
import time

def create_batches(data, batch_size=128):
  batched = []
  for i in range(0, len(data), batch_size):
    b_item = {'text':[], 'positive':[], 'negative':[]}
    for j in range(i, min(i + batch_size, len(data))):
      for key, value in data[j].items():
        b_item[key].append(value)
    batched.append(b_item)
  return batched

In [40]:
split_idx = int(len(train_data)*0.8)

In [41]:
program.train(train_data[:5000],
              valid_set=train_data[split_idx:split_idx+2000],
              test_set=test_data[:2000], train_epoch_num=10, Optim=torch.optim.Adam, device='cuda')

INFO:regr.program.program:Epoch: 1
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.5722), 'negative': tensor(0.5714)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.6830), 'R': tensor(0.6556), 'F1': tensor(0.6690)}, 'negative': {'P': tensor(0.6929), 'R': tensor(0.6809), 'F1': tensor(0.6868)}}
INFO:regr.program.program: - - argmax


INFO:regr.program.program:{'positive': {'P': tensor(0.6811), 'R': tensor(0.6751), 'F1': tensor(0.6781)}, 'negative': {'P': tensor(0.6883), 'R': tensor(0.6907), 'F1': tensor(0.6895)}}
INFO:regr.program.program:Validation:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.3675), 'negative': tensor(0.3682)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8271), 'R': tensor(0.8673), 'F1': tensor(0.8467)}, 'negative': {'P': tensor(0.8646), 'R': tensor(0.8193), 'F1': tensor(0.8414)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8272), 'R': tensor(0.8683), 'F1': tensor(0.8473)}, 'negative': {'P': tensor(0.8649), 'R': tensor(0.8213), 'F1': tensor(0.8425)}}
INFO:regr.program.program:Epoch: 2
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.3420), 'negative': tensor(0.3417)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8525), 'R': tensor(0.8535), 'F1': tensor(0.8530)}, 'negative': {'P': tensor(0.8590), 'R': tensor(0.8533), 'F1': tensor(0.8561)}}
INFO:regr.program.program: - - argmax


...

INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.5465), 'negative': tensor(0.5456)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8587), 'R': tensor(0.8744), 'F1': tensor(0.8665)}, 'negative': {'P': tensor(0.8754), 'R': tensor(0.8598), 'F1': tensor(0.8675)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8587), 'R': tensor(0.8744), 'F1': tensor(0.8665)}, 'negative': {'P': tensor(0.8754), 'R': tensor(0.8598), 'F1': tensor(0.8675)}}
INFO:regr.program.program:Epoch: 7
INFO:regr.program.program:Training:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.0654), 'negative': tensor(0.0652)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.9781), 'R': tensor(0.9769), 'F1': tensor(0.9775)}, 'negative': {'P': tensor(0.9775), 'R': tensor(0.9787), 'F1': tensor(0.9781)}}
INFO:regr.program.program: - - argmax


INFO:regr.program.program:{'positive': {'P': tensor(0.9781), 'R': tensor(0.9769), 'F1': tensor(0.9775)}, 'negative': {'P': tensor(0.9775), 'R': tensor(0.9787), 'F1': tensor(0.9781)}}
INFO:regr.program.program:Validation:


INFO:regr.program.program: - loss:
INFO:regr.program.program:{'positive': tensor(0.6043), 'negative': tensor(0.6047)}
INFO:regr.program.program: - metric:
INFO:regr.program.program: - - ILP
INFO:regr.program.program:{'positive': {'P': tensor(0.8559), 'R': tensor(0.8845), 'F1': tensor(0.8700)}, 'negative': {'P': tensor(0.8837), 'R': tensor(0.8549), 'F1': tensor(0.8690)}}
INFO:regr.program.program: - - argmax
INFO:regr.program.program:{'positive': {'P': tensor(0.8559), 'R': tensor(0.8845), 'F1': tensor(0.8700)}, 'negative': {'P': tensor(0.8828), 'R': tensor(0.8549), 'F1': tensor(0.8686)}}
INFO:regr.program.program:Epoch: 8
INFO:regr.program.program:Training:


KeyboardInterrupt: ignored